In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
transform = transforms.ToTensor()

In [3]:
train_data_set = datasets.MNIST(root='/Users/srisandeepkommanaboyina/Desktop/Deep learning HW!', train=True, download=True, transform=transform)
test_data_set = datasets.MNIST(root='/Users/srisandeepkommanaboyina/Desktop/Deep learning HW!', train=False, download=True, transform=transform)

In [4]:
len(train_data_set)

60000

In [5]:
test_data_set

Dataset MNIST
    Number of datapoints: 10000
    Root location: /Users/srisandeepkommanaboyina/Desktop/Deep learning HW!
    Split: Test
    StandardTransform
Transform: ToTensor()

In [6]:
# Randomly shuffling the labels
rand_labels_train = torch.tensor(np.random.randint(0,10, (len(train_data_set)),))
train_data_set.targets = rand_labels_train
rand_labels_test = torch.tensor(np.random.randint(0,10, (len(test_data_set)),))
test_data_set.targets = rand_labels_test

In [7]:
#Data for training and testing
train_loader = DataLoader(train_data_set, batch_size=50, shuffle=True)
test_loader = DataLoader(test_data_set, batch_size=50, shuffle=True)

In [8]:
class DNModel(nn.Module):
    def __init__(self, inp=784, h1=120, h2=120,h3=16, outp=10):
        super().__init__()
        self.fc1 = nn.Linear(inp,h1)    
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h3, outp)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x

In [9]:
dn_model=DNModel()

In [10]:
dn_model

DNModel(
  (fc1): Linear(in_features=784, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=10, bias=True)
)

In [11]:
CriterionLoss = nn.CrossEntropyLoss()

In [12]:
optim = torch.optim.Adam(dn_model.parameters(), lr=0.001)

In [ ]:
epochs = 4000
train_losses = []
test_losses = []

for i in range(epochs):
    
    #training the batches
    for b, (X_train, y_train) in enumerate(train_loader):
        
        # Applying the model
        y_pred = dn_model(X_train.view(50, -1))  # Here we flatten X_train
        loss = CriterionLoss(y_pred, y_train)
 
        # Updating the parameters
        optim.zero_grad()
        loss.backward()
        optim.step()
        
     
    # Appending train loss  for the epoch
    train_losses.append(loss)
        
    # testing the batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):

            # Applying  the model
            y_val = dn_model(X_test.view(50, -1)) 
    
    # Appending test loss for the epoch
    loss = CriterionLoss(y_val, y_test)
    test_losses.append(loss)

In [ ]:
train_losses=torch.tensor(train_losses)
test_losses=torch.tensor(test_losses)

In [ ]:
# Visulaize losses for for MNIST dataset
plt.plot(range(epochs), train_losses, 'r', label='Train')
plt.plot(range(epochs), test_losses, 'b', label='Test')
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()